In [3]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os


# Load environment variables
load_dotenv()

# Verify that the environment variables are loaded
print(os.getenv('DB_HOST'))
print(os.getenv('DB_NAME'))
print(os.getenv('DB_USER'))
print(os.getenv('DB_PASS'))

# Database connection parameters
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')


def store_prediction(comment, videoid, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist):
    conn = None
    cursor = None
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        cursor = conn.cursor()
        
        # Insert the prediction into the database
        insert_query = sql.SQL("""
            INSERT INTO comment (comment, videoid, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist)
            VALUES (%s, %d, %f, %f, %f, %f, %f, %f, %f)
        """)
        cursor.execute(insert_query, (comment, int(videoid), float(estoxico), float(is_toxic), float(is_abusive), float(is_provocative), float(is_obscene), float(is_hatespeech), float(is_racist)))
        
        # Commit the transaction
        conn.commit()
        
    except Exception as e:
        print(f"Error storing prediction: {e}")
        
    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()

def store_prediction_tab2(video_url, comments):
    conn = None
    cursor = None
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        cursor = conn.cursor()
        
        # Insert the video URL into the database
        insert_video_query = sql.SQL("""
            INSERT INTO video (video_url)
            VALUES (%s)
            RETURNING videoid
        """)
        cursor.execute(insert_video_query, (video_url,))
        video_id = cursor.fetchone()[0]
        
        # Insert each comment prediction into the database
        insert_comment_query = sql.SQL("""
            INSERT INTO comment (videoid, comment, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """)
        for comment in comments:
            cursor.execute(insert_comment_query, (
                video_id, 
                comment['text'], 
                comment['estoxico'], 
                comment['is_toxic'], 
                comment['is_abusive'], 
                comment['is_provocative'], 
                comment['is_obscene'], 
                comment['is_hatespeech'], 
                comment['is_racist']
            ))
        
        # Commit the transaction
        conn.commit()
        
    except Exception as e:
        print(f"Error storing prediction for tab2: {e}")

    finally:
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()


# Example usage for tab1
store_prediction(
    comment="This is a comment from tab1",
    videoid=0,
    estoxico=False,
    is_toxic=0.4,
    is_abusive=0.5,
    is_provocative=0.3,
    is_obscene=0.2,
    is_hatespeech=0.1,
    is_racist=0.0
)

'''
# Example usage for tab2
comments = [
    {
        'text': "This is a comment from tab2",
        'estoxico': False,
        'is_toxic': 0.1,
        'is_abusive': 0.2,
        'is_provocative': 0.3,
        'is_obscene': 0.4,
        'is_hatespeech': 0.5,
        'is_racist': 0.6
    }
]

for comment in comments:
    store_prediction(
        comment=comment['text'],
        estoxico=comment['estoxico'],
        is_toxic=comment['is_toxic'],
        is_abusive=comment['is_abusive'],
        is_provocative=comment['is_provocative'],
        is_obscene=comment['is_obscene'],
        is_hatespeech=comment['is_hatespeech'],
        is_racist=comment['is_racist'],
        video_url="http://example.com/video"
    )

'''

localhost
nlp_youtube
postgres
postgres
Error storing prediction: unsupported format character 'd' (0x64) at index 163


'\n# Example usage for tab2\ncomments = [\n    {\n        \'text\': "This is a comment from tab2",\n        \'estoxico\': False,\n        \'is_toxic\': 0.1,\n        \'is_abusive\': 0.2,\n        \'is_provocative\': 0.3,\n        \'is_obscene\': 0.4,\n        \'is_hatespeech\': 0.5,\n        \'is_racist\': 0.6\n    }\n]\n\nfor comment in comments:\n    store_prediction(\n        comment=comment[\'text\'],\n        estoxico=comment[\'estoxico\'],\n        is_toxic=comment[\'is_toxic\'],\n        is_abusive=comment[\'is_abusive\'],\n        is_provocative=comment[\'is_provocative\'],\n        is_obscene=comment[\'is_obscene\'],\n        is_hatespeech=comment[\'is_hatespeech\'],\n        is_racist=comment[\'is_racist\'],\n        video_url="http://example.com/video"\n    )\n\n'

In [6]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Database connection parameters
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

def store_prediction(comment, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist, video_url=None):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        cursor = conn.cursor()
        
        video_id = None
        if video_url:
            # Insert the video URL into the video table
            insert_video_query = """
                INSERT INTO video (videourl)
                VALUES (%s)
                RETURNING videoid
            """
            cursor.execute(insert_video_query, (video_url,))
            video_id = cursor.fetchone()[0]
        
        # Insert the prediction into the comment table
        insert_comment_query = """
            INSERT INTO comment (videoid, comment, es_toxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_comment_query, (video_id, comment, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist))
        
        # Commit the transaction
        conn.commit()
        
    except Exception as e:
        print(f"Error storing prediction: {e}")
    finally:
        cursor.close()
        conn.close()

# Example usage for tab1
store_prediction(
    comment="This is a comment from tab1",
    estoxico=True,
    is_toxic=0.8,
    is_abusive=0.5,
    is_provocative=0.3,
    is_obscene=0.2,
    is_hatespeech=0.1,
    is_racist=0.0
)

# Example usage for tab2
comments = [
    {
        'text': "This is a comment from tab2",
        'estoxico': False,
        'is_toxic': 0.1,
        'is_abusive': 0.2,
        'is_provocative': 0.3,
        'is_obscene': 0.4,
        'is_hatespeech': 0.5,
        'is_racist': 0.6
    }
]

for comment in comments:
    store_prediction(
        comment=comment['text'],
        estoxico=comment['estoxico'],
        is_toxic=comment['is_toxic'],
        is_abusive=comment['is_abusive'],
        is_provocative=comment['is_provocative'],
        is_obscene=comment['is_obscene'],
        is_hatespeech=comment['is_hatespeech'],
        is_racist=comment['is_racist'],
        video_url="http://example2.com/video"
    )

In [9]:
import psycopg2
from psycopg2 import sql
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Database connection parameters
DB_HOST = os.getenv('DB_HOST')
DB_NAME = os.getenv('DB_NAME')
DB_USER = os.getenv('DB_USER')
DB_PASS = os.getenv('DB_PASS')

def store_prediction(comment, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist, video_url=None):
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS)
        cursor = conn.cursor()
        
        video_id = None
        if video_url:
            # Check if the video URL already exists
            check_video_query = """
                SELECT videoid FROM video WHERE videourl = %s
            """
            cursor.execute(check_video_query, (video_url,))
            result = cursor.fetchone()
            
            if result:
                video_id = result[0]
            else:
                # Insert the video URL into the video table
                insert_video_query = """
                    INSERT INTO video (videourl)
                    VALUES (%s)
                    RETURNING videoid
                """
                cursor.execute(insert_video_query, (video_url,))
                video_id = cursor.fetchone()[0]
        
        # Check if the comment + videoid combination already exists
        check_comment_query = """
            SELECT comment_id FROM comment WHERE comment = %s AND videoid = %s
        """
        cursor.execute(check_comment_query, (comment, video_id))
        result = cursor.fetchone()
        
        if not result:
            # Insert the prediction into the comment table
            insert_comment_query = """
                INSERT INTO comment (videoid, comment, es_toxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(insert_comment_query, (video_id, comment, estoxico, is_toxic, is_abusive, is_provocative, is_obscene, is_hatespeech, is_racist))
            conn.commit()
        
    except Exception as e:
        print(f"Error storing prediction: {e}")
    finally:
        cursor.close()
        conn.close()

# Example usage for tab1
store_prediction(
    comment="This is a comment from tab1",
    estoxico=True,
    is_toxic=0.8,
    is_abusive=0.5,
    is_provocative=0.3,
    is_obscene=0.2,
    is_hatespeech=0.1,
    is_racist=0.0
)

# Example usage for tab2
comments = [
    {
        'text': "This is a comment from tab2",
        'estoxico': False,
        'is_toxic': 0.1,
        'is_abusive': 0.2,
        'is_provocative': 0.3,
        'is_obscene': 0.4,
        'is_hatespeech': 0.5,
        'is_racist': 0.6
    }
]

for comment in comments:
    store_prediction(
        comment=comment['text'],
        estoxico=comment['estoxico'],
        is_toxic=comment['is_toxic'],
        is_abusive=comment['is_abusive'],
        is_provocative=comment['is_provocative'],
        is_obscene=comment['is_obscene'],
        is_hatespeech=comment['is_hatespeech'],
        is_racist=comment['is_racist'],
        video_url="http://example.com/video"
    )